# 3: Detect and align confocal / STED spots, calculate distances

0) Split aligend DNA and RNA `tif` images into single channels. 
2) Detect RNA and DNA FISH spots in same image (confocal or STED), using RS-FISH settings files as input, outputing `merge_subpixel.csv` list of spots
3) Match DNA and RNA spots and calculate distances

For more details about required in and output please look at the individual notebooks used in the pipeline.

In [ ]:
import os
import queue
import pandas as pd
from pathlib import Path
from datetime import datetime
import papermill as pm
from multiprocessing import Pool
import concurrent.futures

def run_notebook(parameters,notebook_to_run,parameters_common={}):
    
    # change to directory where the notebook is (resolve relative imports)
    os.chdir(Path(notebook_to_run).absolute().parent)
    
    # run notebook
    for parameters_spec in parameters_list:
        parameters = {**parameters_common, **parameters_spec}

        pm.execute_notebook(
           notebook_to_run,
           '/dev/null',
           parameters=parameters)

# 0) Split tifs into individual channels

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"}
]

parameters_common = {"tif_path_suffix": "aligned_confocal",      # tif folder suffix relative to in_path
                     "out_path_suffix": "tif_aligned_confocal"}  # out folder suffix relative to out_path

notebook_to_run = "/home/stumberger/ep2024/subscripts/resave_tif_as_tif_split_channel.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 1) Spot detection DNA and RNA
**In:** 
- `tif_subfolder` containing single channel `tif` images
- `out_subfolder` containing spot detection settings file `chX.txt` generated via RS-FISH Fiji plugin for each channel specified in `channels`
- `acquisition_info.json` file with the sample metadata which you wish to be added to your spots

**Out:**
- folder containing a `csv` file for each provioded `tif`
- `merge_subpixel.csv` file combining spots and their metadata from all images in the batch
- vis folder with detection visualization for all `tif`s as `png` (good for checking detection results)

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"}
]

parameters_common = {"channels": [0,1,2],
                     "tif_subfolder": "tif_aligned_confocal",
                     "out_subfolder": "detections_confocal1"}

notebook_to_run = "/home/stumberger/ep2024/subscripts/RS-FISH_spot_detection.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 2) Calculate DNA-DNA and RNA-DNA distances

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"} # out path
]

### common parameters
parameters_common = {
     "rel_spot_path": "/detections_confocal1/merge_subpixel.csv", # spot file path relative to in_path1
     "channels_ep": [0,1], # E - P channels,
     "channels_pr": [0,2], # P - RNA channels,
    "out": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/distances_confocal_subpixel.csv"
}

notebook_to_run = "/home/stumberger/ep2024/subscripts/paired_spot_distances_RNA_DNA.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)